In [ ]:
import os
import tensorflow as tf
from tensorflow import keras
import numpy as np
import tempfile
import shutil

# print the tensor flow version
print(tf.__version__)

In [ ]:
from utils import display_digit
from utils import display_digit_v1
from utils import display_digit_v2
from utils import unfold_labels

num_classes = 10
batch_size = 100

In [ ]:
# setup tensorboard
tensorboard_file_name = tempfile.gettempdir() + "/TensorFlow/tensor_flow_basic"
shutil.rmtree(tensorboard_file_name, ignore_errors = True) # start fresh each time
writer = tf.summary.FileWriter(tensorboard_file_name)

In [ ]:
# load data
#from tensorflow.examples.tutorials.mnist import input_data
#mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
#mnist = tf.keras.datasets.mnist
#(x_train, y_train), (x_test, y_test) = mnist.load_data()
#x_train = x_train.astype(np.float32) / 255
#x_train = np.expand_dims(x_train, -1)
#x_train = x_train.astype(np.float32)
#x_train = x_train.reshape([-1, 784])
#y_train = tf.one_hot(y_train, num_classes)

#mnist = tf.data.Dataset.from_tensor_slices((x_train, y_train))
#iter = mnist.batch(batch_size).make_one_shot_iterator()

# Code from https://stackoverflow.com/questions/50666681/how-to-load-mnist-via-tensorflow-including-download
#load mnist data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

def create_mnist_dataset(data, labels, batch_size):
  def gen():
    for image, label in zip(data, labels):
        yield image, label
  ds = tf.data.Dataset.from_generator(gen, (tf.float32, tf.int32), ((28,28 ), ()))

  return ds.repeat().batch(batch_size)

#train and validation dataset with different batch size
training_dataset = create_mnist_dataset(x_train, y_train, 10)
validation_dataset = create_mnist_dataset(x_test, y_test, 20)

#y_train = tf.one_hot(y_train, num_classes)

handle = tf.placeholder(tf.string, shape=[])
iterator = tf.data.Iterator.from_string_handle(handle, 
                                               training_dataset.output_types, 
                                               validation_dataset.output_shapes)
image, label = iterator.get_next()

train_iterator = training_dataset.make_one_shot_iterator()
valid_iterator = validation_dataset.make_one_shot_iterator()

In [ ]:
training_set_size = y_train.shape[0]

# define the graph
# z = W*x + b
# y = softmax(z)
#x = tf.placeholder(tf.float32, [None, 784], name = "X")
#W = tf.Variable(tf.zeros([784, 10]), name = "W")
#b = tf.Variable(tf.zeros([10]), name = "b")
#z = tf.add(tf.matmul(x, W), b, name = "z")
#y = tf.nn.softmax(z, name = "y")
#y_ = tf.placeholder(tf.float32, [None, 10], name = "y_")
#y_ = y_train

In [ ]:
#cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]), name = "cross_entropy")
#train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [ ]:
#batch_xs, batch_ys = iter.get_next()
print(y_train.shape)

In [ ]:
handle = tf.placeholder(tf.string, shape=[])
iterator = tf.data.Iterator.from_string_handle(handle, 
                                               training_dataset.output_types, 
                                               training_dataset.output_shapes)
image, label = iterator.get_next()

train_iterator = training_dataset.make_one_shot_iterator()
valid_iterator = validation_dataset.make_one_shot_iterator()

y = tf.layers.dense(tf.layers.flatten(image),1,activation=tf.nn.relu)
loss = tf.losses.mean_squared_error(tf.squeeze(y), label)

with tf.Session() as sess:
    # write the graph to the tensorboard log
    writer.add_graph(sess.graph)
    
    tf.global_variables_initializer().run()

    # The `Iterator.string_handle()` method returns a tensor that can be evaluated
    # and used to feed the `handle` placeholder.
    train_handle = sess.run(train_iterator.string_handle())
    valid_handle = sess.run(valid_iterator.string_handle())

    # Run training
    train_loss, train_img, train_label = sess.run([loss, image, label],
                                                 feed_dict={handle: train_handle})
    # train_image.shape = (10, 784) 

    # Run validation
    valid_pred, valid_img = sess.run([y, image], 
                                    feed_dict={handle: valid_handle})


In [ ]:
#print(accuracy.eval(session=tf.Session()))

In [ ]:
#labels = unfold_labels(mnist.test.labels)
#errors = np.where(pred != labels)[0]

#random_error_indx = errors[np.random.randint(errors.size, size=1)[0]]
#display_digit(mnist.test, random_error_indx, pred[random_error_indx])

In [ ]:
iterator = training_dataset.make_one_shot_iterator()
next_element = iterator.get_next()

with tf.Session() as sess:
    image = sess.run(next_element)
    display_digit_v1(image, 5, 6)
    display_digit_v2(x_train, y_train, 6)

In [ ]:
tensorboard_file_name